<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1c9f743efab7804ca4e7fd12e64a03d682fc9ed66ef51c012d2e0a32c970ed46
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 12:13:04
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 30.72 K (0.22%)
Current PnL: -16.73 L (-11.26%)
CY Booked + Current PnL: -5.29 L (-3.56%)
-------------------
Total profit:  2.06 L
Total loss:  -18.80 L
-------------------
Total Booked + Current PnL: 21.69 L (17.82%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.10 L (57.46%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,104.06,56.0,M-SC,3.98,87940.0,-12837.0,12927.0,-0.81,-12.74,14.70,0.09,245.0,-0.99,0.64,15.53,OX40N,NTT,DURABLES
56,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,220787.0,10421.0,18369.0,0.11,4.95,8.32,13.68,37.0,0.57,1.62,22.30,XY25,NTT,REFINERIES
49,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112546.0,12042.0,18593.0,-0.43,11.98,16.52,30.49,79.0,0.65,0.82,48.27,MH,ATH,METALS
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
48,MASFIN,398.61,-16.44,53.0,H-SC,12.99,94740.0,-3240.0,24841.0,-0.17,-3.31,26.22,22.05,152.0,-0.13,0.69,37.71,XR,ATH,FINANCE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,62.0,M-MC,6.95,226960.0,1998.0,165658.0,-0.72,0.89,72.99,74.53,192.0,0.01,1.66,33.43,XY24,BTT,STEEL
35,IEX,219.00,-29.93,54.0,H-SC,17.09,200293.0,339.0,96882.0,-0.57,0.17,48.37,48.63,115.0,0.00,1.47,11.74,XR,NTT,MISC
29,HAVELLS,2069.16,-0.09,57.0,H-MC,1.86,247728.0,-3.0,75062.0,-0.35,-0.00,30.30,30.30,92.0,-0.00,1.81,13.29,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200745.0,-75.0,68856.0,-0.01,-0.04,34.30,34.25,80.0,-0.00,1.47,5.16,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.03,39.0,H-LC,6.38,298712.0,-17130.0,129790.0,-0.60,-5.42,43.45,35.67,5.0,-0.13,2.19,7.68,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.06,47.0,H-LC,7.83,219498.0,-32270.0,85473.0,0.03,-12.82,38.94,21.13,27.0,-0.38,1.61,18.07,X40,ATH,PAINTS
79,UNITDSPR,1644.00,-12.94,51.0,H-LC,7.80,228055.0,-6089.0,54710.0,-0.84,-2.60,23.99,20.77,86.0,-0.11,1.67,4.14,X40N,NTT,BREWERIES
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
43,JIOFIN,387.00,0.31,52.0,H-LC,13.22,272448.0,5083.0,60374.0,0.30,1.90,22.16,24.48,48.0,0.08,1.99,57.92,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.87,72.0,H-LC,1.30,250448.0,4297.0,74909.0,2.63,1.75,29.91,32.18,16.0,0.06,1.83,38.42,X200,ATH,IT
41,ITC,452.0,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.10,58.0,H-LC,3.86,282836.0,17410.0,39371.0,0.37,6.56,13.92,21.40,11.0,0.44,2.07,13.64,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,220787.0,10421.0,18369.0,0.11,4.95,8.32,13.68,37.0,0.57,1.62,22.30,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.78,214871.0,10371.0,31049.0,0.56,5.07,14.45,20.25,10.0,0.33,1.57,14.25,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,0.87,72.0,H-LC,1.30,250448.0,4297.0,74909.0,2.63,1.75,29.91,32.18,16.0,0.06,1.83,38.42,X200,ATH,IT
7,ATULAUTO,844.00,3943.24,65.0,M-SC,5.80,126024.0,-21344.0,73157.0,1.98,-14.48,58.05,35.16,236.0,-0.29,0.92,28.72,XY24,NTT,AUTO
60,SAMMAANCAP,326.00,-171.59,63.0,M-SC,2.63,82512.0,-19698.0,113091.0,1.81,-19.27,137.06,91.37,208.0,-0.17,0.60,33.94,XY25,NTT,FINANCE
39,INFY,2275.00,-15.29,59.0,H-LC,8.39,329894.0,17006.0,154687.0,1.73,5.44,46.89,54.87,3.0,0.11,2.41,12.45,X40,BTT,IT
15,BSOFT,836.99,-12.08,57.0,H-SC,15.02,98787.0,-41866.0,114642.0,1.68,-29.77,116.05,51.74,131.0,-0.37,0.72,12.65,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJOOENG,143.33,-36.25,38.0,H-SC,11.59,96620.0,-5880.0,46706.0,-1.41,-5.74,48.34,39.83,114.0,-0.13,0.71,0.99,AR,ATH,MISC
31,HINDZINC,730.22,20.84,58.0,M-LC,1.35,197055.0,-8021.0,120598.0,-1.40,-3.91,61.20,54.89,52.0,-0.07,1.44,19.24,X5K,ATH,METALS
45,KPIGREEN,731.05,1.73,41.0,H-SC,10.00,121300.0,-3997.0,62918.0,-1.17,-3.19,51.87,47.03,141.0,-0.06,0.89,51.88,MH,ATH,POWER
26,GREENPANEL,537.00,234.63,63.0,M-SC,2.41,149499.0,-33579.0,112558.0,-1.16,-18.34,75.29,43.14,230.0,-0.30,1.09,40.62,XY24,NTT,MISC
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.74,89191.0,-11772.0,106307.0,-0.94,-11.66,119.19,93.63,148.0,-0.11,0.65,32.85,SR,ATH,CHEMICALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,104.06,56.0,M-SC,3.98,87940.0,-12837.0,12927.0,-0.81,-12.74,14.70,0.09,245.0,-0.99,0.64,15.53,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.53,123903.0,-18576.0,38100.0,0.31,-13.04,30.75,13.70,163.0,-0.49,0.91,46.24,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,64.0,H-SC,1.39,227430.0,-42237.0,78577.0,1.29,-15.66,34.55,13.47,138.0,-0.54,1.66,15.03,XY24,NTT,PAINTS
18,CERA,9475.0,-19.33,51.0,H-SC,2.08,147085.0,-28818.0,70836.0,0.33,-16.38,48.16,23.89,149.0,-0.41,1.08,27.03,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.66,108614.0,-23221.0,65451.0,0.59,-17.61,60.26,32.03,98.0,-0.35,0.79,21.66,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,59.0,H-LC,12.33,298262.0,-47694.0,122556.0,0.01,-13.79,41.09,21.64,1.0,-0.39,2.18,5.66,X40,ATH,IT
39,INFY,2275.00,-15.29,59.0,H-LC,8.39,329894.0,17006.0,154687.0,1.73,5.44,46.89,54.87,3.0,0.11,2.41,12.45,X40,BTT,IT
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
82,VBL,671.64,-17.03,39.0,H-LC,6.38,298712.0,-17130.0,129790.0,-0.60,-5.42,43.45,35.67,5.0,-0.13,2.19,7.68,X40N,ATH,FMCG
1,ABB,7934.00,-37.73,67.0,H-LC,10.34,260458.0,-1161.0,120384.0,0.75,-0.44,46.22,45.57,7.0,-0.01,1.91,10.83,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,62.0,M-MC,6.95,226960.0,1998.0,165658.0,-0.72,0.89,72.99,74.53,192.0,0.01,1.66,33.43,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,13.92,65.0,H-MC,3.58,196670.0,32000.0,35971.0,1.37,19.43,18.29,41.28,89.0,0.89,1.44,28.97,X40N,ATH,AC
83,VOLTAS,1918.49,1.86,61.0,H-MC,2.51,213750.0,22008.0,74022.0,0.93,11.48,34.63,50.08,99.0,0.30,1.56,19.06,XY25,ATH,AC
17,CAMS,5211.76,0.38,67.0,H-SC,3.81,114156.0,12152.0,31770.0,1.58,11.91,27.83,43.06,122.0,0.38,0.84,31.07,X40N,ATH,MISC
23,DABUR,735.00,-4.38,51.0,H-MC,6.70,208065.0,11037.0,78586.0,-0.31,5.60,37.77,45.49,102.0,0.14,1.52,19.37,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112546.0,12042.0,18593.0,-0.43,11.98,16.52,30.49,79.0,0.65,0.82,48.27,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,50.0,M-SC,0.47,100005.0,8506.0,70244.0,-0.61,9.30,70.24,86.07,223.0,0.12,0.73,45.23,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.03,41.0,H-SC,11.66,133931.0,10595.0,123163.0,-0.93,8.59,91.96,108.45,119.0,0.09,0.98,32.86,AR,ATH,MISC
85,WIPRO,420.00,-9.71,59.0,M-LC,5.63,158900.0,7955.0,101505.0,0.84,5.27,63.88,72.51,53.0,0.08,1.16,11.61,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,64.0,H-MC,6.12,183437.0,3629.0,108778.0,-0.31,2.02,59.30,62.52,88.0,0.03,1.34,36.79,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1896.21,67.0,L-SC,3.05,54471.0,-82706.0,82801.0,0.35,-60.29,152.01,0.07,267.0,-1.00,0.40,35.47,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,6700.00,52.0,L-SC,5.18,74603.0,-19207.0,95828.0,0.69,-20.47,128.45,81.67,269.0,-0.20,0.55,44.47,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.40,85920.0,-27629.0,67679.0,0.95,-24.33,78.77,35.27,268.0,-0.41,0.63,110.29,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.74,89191.0,-11772.0,106307.0,-0.94,-11.66,119.19,93.63,148.0,-0.11,0.65,32.85,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,53.0,H-SC,12.99,94740.0,-3240.0,24841.0,-0.17,-3.31,26.22,22.05,152.0,-0.13,0.69,37.71,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3943.24,65.0,M-SC,5.80,126024.0,-21344.0,73157.0,1.98,-14.48,58.05,35.16,236.0,-0.29,0.92,28.72,XY24,NTT,AUTO
21,COFFEEDAY,80.0,-41.65,53.0,L-SC,42.40,85920.0,-27629.0,67679.0,0.95,-24.33,78.77,35.27,268.0,-0.41,0.63,110.29,XR,NTT,HOTELS
80,VAIBHAVGBL,521.0,67.49,72.0,H-SC,7.21,145004.0,-37771.0,148832.0,1.55,-20.67,102.64,60.77,125.0,-0.25,1.06,29.87,XR,NTT,JEWELLERY
60,SAMMAANCAP,326.0,-171.59,63.0,M-SC,2.63,82512.0,-19698.0,113091.0,1.81,-19.27,137.06,91.37,208.0,-0.17,0.60,33.94,XY25,NTT,FINANCE
53,RAJESHEXPO,518.0,1896.21,67.0,L-SC,3.05,54471.0,-82706.0,82801.0,0.35,-60.29,152.01,0.07,267.0,-1.00,0.40,35.47,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.10
1,25,43.88
2,50,74.46


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.58
LC    32.15
MC    23.25
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.27
X40      15.44
X40N     11.96
XR       11.77
AR        8.76
XY25      8.27
OX40N     8.23
X200      1.83
MH        1.71
X5K       1.44
SR        1.30
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.36
H-LC    25.45
H-MC    20.29
M-SC    13.64
M-LC     5.63
M-MC     2.65
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.68,-12.27,73.49
FMCG,11.88,-4.24,40.71
FINANCE,9.76,-15.19,60.30
MISC,7.50,-12.23,74.64
BANKS,6.32,-15.68,76.53
PAINTS,5.88,-14.88,32.01
ELECTRICAL,5.76,-4.52,62.13
AC,3.72,4.89,27.33
INSURANCE,3.44,-6.93,43.61


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2840927.0,22
XR,1301112.0,14
AR,1299704.0,9
X40,737428.0,10
X40N,596855.0,9
OX40N,520910.0,10
XY25,392023.0,6
SR,243800.0,2
X5K,120598.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3399483.0,29
M-SC,1378263.0,17
H-LC,1214584.0,14
H-MC,1177919.0,15
M-LC,379369.0,4
M-MC,315563.0,2
L-SC,246308.0,3
L-MC,60559.0,1
L-LC,37729.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1021460.0      6
M-SC       XY24       794972.0      7
H-SC       AR         786767.0      5
           XR         751985.0      7
H-MC       XY24       491292.0      4
H-LC       X40        454005.0      5
           AR         352007.0      2
M-MC       XY24       315563.0      2
H-SC       X40N       284416.0      4
           OX40N      248137.0      4
           SR         243800.0      2
H-LC       X40N       215549.0      3
H-MC       X40        213452.0      4
           XY25       183463.0      2
L-SC       XR         163507.0      2
M-SC       AR         160930.0      2
M-LC       XY24       157266.0      2
M-SC       OX40N      124521.0      4
M-LC       X5K        120598.0      1
M-SC       XR         114778.0      2
           XY25       113091.0      1
H-MC       XR         108778.0      1
M-LC       XR         101505.0      1
H-MC       X40N        96890.0      2
L-SC       OX40N       82801.0      1
H-LC       X200        74909.0      1
M-SC       X40         69971.0      1
H-MC       OX40N       65451.0      1
H-SC       MH          62918.0      1
L-MC       XR          60559.0      1
H-LC       XY24        60374.0      1
           XY25        57740.0      2
L-LC       XY25        37729.0      1
H-MC       MH          18593.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
